# Imports

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [3]:
import talos as ta
from talos.model import lr_normalizer, early_stopper, hidden_layers

import tensorflow as tf
if tf.test.is_gpu_available() & tf.test.is_built_with_cuda():
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    #from tensorflow.python.client import device_lib
    #print(device_lib.list_local_devices())

#Original: from keras import callbacks, backend as K
from tensorflow.compat.v1.keras import callbacks, backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, MobileNetV2
from keras.utils import multi_gpu_model
from keras.initializers import glorot_uniform
from keras.optimizers import Adam, Nadam, RMSprop, SGD, Adagrad
from keras.layers.advanced_activations import ReLU, LeakyReLU

from datetime import datetime
import pandas as pd
import numpy as np

from numpy.random import seed
seed(1)
#Original: from tensorflow import set_random_seed
#Original: set_random_seed(1)
tf.random.set_seed(1)

#Original config = tf.ConfigProto()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
#Original sess = tf.Session(config=config)
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

The installed version of TensorFlow 2.0.0 includes GPU support.



# Hilfsfunktionen

### Benutzerdefinierte Kostenfunktion & Metrik

In [4]:
def circular_mse(y_true, y_pred):
    max_error = tf.constant(360, dtype='float32')
    return K.mean(K.square(K.minimum(K.abs(y_pred - y_true), max_error - K.abs(y_pred - y_true))), axis=-1)

def circular_mae(y_true, y_pred):
    max_error = tf.constant(360, dtype='float32')
    return K.mean(K.minimum(K.abs(y_pred - y_true), K.abs(max_error - K.abs(y_pred - y_true))), axis=-1)

### Generierung Bottleneck-Features

In [5]:
def create_bottleneck_features(train_generator, valid_generator, bottleneck):
        
    print('Creating bottleneck features...')
    model = MobileNetV2(include_top=False, weights='imagenet')
        
    bottleneck_features_train = model.predict_generator(train_generator, train_generator.n//train_generator.batch_size)
    np.save(open(_LOG_DIR + 'Train_' + bottleneck, 'wb'), bottleneck_features_train)
        
    bottleneck_features_valid = model.predict_generator(valid_generator, valid_generator.n//valid_generator.batch_size)
    np.save(open(_LOG_DIR + 'Valid_'  + bottleneck, 'wb'), bottleneck_features_valid)

### Generierung Datenpipeline (Angepasst für Talos)

In [6]:
def prepare_data(batch_size, num_samples):

    df = pd.read_csv(_CSV_FILE)
    df_shuffled = df.sample(frac=1, random_state=1)
    df_train = df_shuffled[0:int(num_samples*0.8//batch_size*batch_size)]   
    df_valid = df_shuffled.drop(df_shuffled.index[0:df_train.shape[0]])[0:int(num_samples*0.2//batch_size*batch_size)]
    
    train_labels = df_train.drop(['Filename'], axis=1).values
    valid_labels = df_valid.drop(['Filename'], axis=1).values
    
    bottleneck = 'Bottleneck_Features_{}_{}.npy'.format(str(num_samples), str(batch_size))                                       
    if not os.path.exists(_LOG_DIR + 'Train_' + bottleneck):
        
        if _USE_DATA_AUGMENTATION:
            train_data_generator = ImageDataGenerator(
                rescale=1./255
                ,width_shift_range=0.1
                ,height_shift_range=0.1
                ,zoom_range=0.1
                ,brightness_range=(0.25, 0.75)
                ,fill_mode='nearest'
            )
        else:
            train_data_generator = ImageDataGenerator(
                rescale=1./255
            )
            
        valid_data_generator = ImageDataGenerator(rescale=1./255)
    
        train_generator = train_data_generator.flow_from_dataframe(
            dataframe=df_train
            ,directory=_IMAGE_DIR
            ,x_col='Filename'
            ,y_col=['Elevation', 'Azimuth']
            ,class_mode='other'
            ,target_size=(224, 224)
            ,color_mode='rgb'
            ,shuffle=False
            ,seed=1
            ,batch_size=batch_size
        )
    
        valid_generator = valid_data_generator.flow_from_dataframe(
            dataframe=df_valid
            ,directory=_IMAGE_DIR
            ,x_col='Filename'
            ,y_col=['Elevation', 'Azimuth']
            ,class_mode='other'
            ,target_size=(224, 224)
            ,color_mode='rgb'
            ,shuffle=False
            ,seed=1
            ,batch_size=batch_size
        )
                                                        
        create_bottleneck_features(train_generator, valid_generator, bottleneck)
        
    train_features = np.load(open(_LOG_DIR + 'Train_' + bottleneck, 'rb'))
    valid_features = np.load(open(_LOG_DIR + 'Valid_' + bottleneck, 'rb'))
                                                        
    return train_labels, valid_labels, train_features, valid_features

### Generierung Modell (Angepasst für Talos)

In [7]:
def grid_model_base(x, y, x_val, y_val, params):
    
    global _COUNTER    
    K.clear_session()

    train_labels, valid_labels, train_features, valid_features = prepare_data(params['batch_size'], params['samples'])

    model = Sequential()
    model.add(Flatten(input_shape=train_features.shape[1:])) # (7, 7, 512)
    
    dropout_rate = params['dropout']
    first_neuron = params['first_neuron']
    
    if params['activation'] == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = 0.1)
    elif params['activation'] == 'relu':
        activation_layer = ReLU()  
    
    model.add(Dense(units=first_neuron, kernel_initializer=glorot_uniform(seed=1)))
    model.add(activation_layer)
    if dropout_rate > 0.0:
        model.add(Dropout(rate=dropout_rate))
        
    hidden_neuron_fraction = first_neuron
    for i in range(params['hidden_layers']):
        hidden_neuron_fraction = hidden_neuron_fraction//2
        model.add(Dense(units=hidden_neuron_fraction, kernel_initializer=glorot_uniform(seed=1)))
        model.add(activation_layer)
        if dropout_rate > 0.0:
            model.add(Dropout(rate=dropout_rate))
    
    model.add(Dense(units=2, kernel_initializer=glorot_uniform(seed=1)))

    model.compile(optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])),
                  loss=circular_mse if(_LOSS == 'CMSE') else 'mean_squared_error', metrics=[circular_mae])
    
    checkpointer = callbacks.ModelCheckpoint(filepath=_LOG_DIR + 'Best_Weights_FC_{}.hdf5'.format(_COUNTER)
                                             ,monitor='val_circular_mae'
                                             #,monitor='val_loss'
                                             ,verbose=1
                                             ,save_best_only=True
                                             ,mode='min')
    
    startTime = datetime.now()
    out = model.fit(x=train_features
                    ,y=train_labels
                    ,epochs=params['epochs']
                    ,validation_data=(valid_features, valid_labels)
                    ,steps_per_epoch=int(params['samples']*0.8//params['batch_size'])
                    ,validation_steps=int(params['samples']*0.2//params['batch_size'])
                    ,callbacks=[checkpointer]
                   )
    print("Time taken:", datetime.now() - startTime)
    
    _COUNTER = _COUNTER + 1
     
    return out, model

# Parameter

### Dateisystem

In [8]:
_RUN = 'tmp_MobileNetV2'
_LOSS = 'MSE'
_DATASET_NAME = 'Dataset_2019-08-13'

In [9]:
_USE_DATA_AUGMENTATION = False

In [10]:
_MODEL_NAME = '{}_Optimierung_Hyperparameter_v{}'.format(_DATASET_NAME, _RUN)
#Original: _IMAGE_DIR = 'D:\\tkroiss\\Datasets\\{}\\'.format(_DATASET_NAME)
_IMAGE_DIR = '..\\data-target\\Datasets\\{}\\'.format(_DATASET_NAME)
_CSV_FILE = _IMAGE_DIR + 'images.csv'

_COUNTER = 0
_DEVICE = 'RTX' if(os.environ["CUDA_VISIBLE_DEVICES"]=='0') else 'TITAN'

#Original: _LOG_DIR = 'E:\\tkroiss\\{}_Regression_{}\\Base\\'.format(_RUN, _LOSS)
_LOG_DIR = '..\\logs\\{}_Regression_{}\\Base\\'.format(_RUN, _LOSS)
assert(os.path.exists(_LOG_DIR)) == False, 'Vorsicht, Verzeichnis existiert bereits!'
os.makedirs(_LOG_DIR)

### GridSerach

In [11]:
def get_params():
    
#     Adam = RMSprop + Momentum (lr=0.001)
#     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
#     RMSprop = (lr=0.001)
#     SGD = (lr=0.01)
    
    p = {'samples':[20000],
         'epochs':[1],
         'batch_size':[64, 128],
         'optimizer':[Adam, Nadam, Adagrad, RMSprop],
         'lr':[1, 2, 5],
         'first_neuron':[1024, 2048, 4096],
         'dropout':[0.25, 0.50],
         'activation':['leakyrelu', 'relu'],
         'hidden_layers':[0, 1, 2, 3, 4]}
    return p

In [12]:
def get_params():
    
#     Adam = RMSprop + Momentum (lr=0.001)
#     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
#     RMSprop = (lr=0.001)
#     SGD = (lr=0.01)
    
    p = {'samples':[20000],
         'epochs':[1],
         'batch_size':[64],
         'optimizer':[Adam],
         'lr':[1],
         'first_neuron':[1024],
         'dropout':[0.25],
         'activation':['leakyrelu'],
         'hidden_layers':[2]}
    return p

# Ausführung GridSearch mit Talos

In [13]:
import numpy as np
dummy_x = np.empty((1, 2, 3, 224, 224))
dummy_y = np.empty((1, 2))

with tf.device('/device:GPU:0'):

        t = ta.Scan(
            x=dummy_x
            ,y=dummy_y
            ,model=grid_model_base
            ,params=get_params()
            ,experiment_name= 'mse_base_best_hp'
            #,shuffle=False
            ,reduction_metric = 'val_circular_mae'
            ,disable_progress_bar=False
            ,print_params = True
            ,clear_session=True
        )

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

{'activation': 'leakyrelu', 'batch_size': 64, 'dropout': 0.25, 'epochs': 1, 'first_neuron': 1024, 'hidden_layers': 2, 'lr': 1, 'optimizer': <class 'keras.optimizers.Adam'>, 'samples': 20000}


ValueError: Invalid class_mode: other; expected one of: {'categorical', 'multi_output', None, 'raw', 'sparse', 'input', 'binary'}